In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, find_similar_artists, find_top_tracks, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [7]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
llm_response = llm_with_tools.invoke([system_message, human_message])

2024-11-14 22:07:12,572 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [10]:
llm_response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_8Expmq22UFTWlgBZfqJ0YKct)
 Call ID: call_8Expmq22UFTWlgBZfqJ0YKct
  Args:
    plan: {'steps': [{'step_number': 1, 'name': 'retrieve-original-playlist-artists', 'type': 'action', 'description': "Get a list of unique artists from the 'New Rock and Blues' playlist.", 'success_criteria': 'Successfully retrieve a list of artists from the specified playlist.', 'tool': 'functions.get_artists_from_playlist', 'action': "Call the function to retrieve artists from 'New Rock and Blues' playlist.", 'condition': None, 'substeps': None}, {'step_number': 2, 'name': 'find-similar-artists', 'type': 'loop', 'description': 'For each artist in the retrieved list, find 3-4 similar artists.', 'success_criteria': 'Identify at least 3 similar artists for each artist in the original list.', 'tool': 'functions.find_similar_artists', 'action': None, 'condition': 'For each artist in the original list,

In [11]:
plan = llm_response.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'step_number': 1,
    'name': 'retrieve-original-playlist-artists',
    'type': 'action',
    'description': "Get a list of unique artists from the 'New Rock and Blues' playlist.",
    'success_criteria': 'Successfully retrieve a list of artists from the specified playlist.',
    'tool': 'functions.get_artists_from_playlist',
    'action': "Call the function to retrieve artists from 'New Rock and Blues' playlist.",
    'condition': None,
    'substeps': None},
   {'step_number': 2,
    'name': 'find-similar-artists',
    'type': 'loop',
    'description': 'For each artist in the retrieved list, find 3-4 similar artists.',
    'success_criteria': 'Identify at least 3 similar artists for each artist in the original list.',
    'tool': 'functions.find_similar_artists',
    'action': None,
    'condition': 'For each artist in the original list, execute the sub-steps.',
    'substeps': [{'step_number': 1,
      'name': 'get-similar-artists',
      'type': 'action',
    

# chat_prompt_template holds all messages (Human, AI, Tool)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Check for Tools in Message

In [13]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

[StructuredTool(name='validate_plan', description='Validates a step-by-step plan to solve a problem\n\nArgs:\n    plan (Plan): a step-by-step plan to solve a problem\n\nReturns:\n    bool: whether plan is okay or not', args_schema=<class 'langchain_core.utils.pydantic.validate_plan'>, func=<function validate_plan at 0x0000018D0E0FFF60>)]


# Tool Call (should be validate_plan())

In [14]:
tool_response = tool_node.invoke({"messages": messages})
tool_message = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: validate_plan

true


# Add Tool Response to Messages

In [15]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Messages with Tool result to LLM

In [16]:
llm_response = llm_with_tools.invoke(chat_prompt_template.format_messages())
llm_response.pretty_print()

2024-11-14 22:07:13,620 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_playlists (call_8QiYNgN1pL5bmd5e0l9tTrXs)
 Call ID: call_8QiYNgN1pL5bmd5e0l9tTrXs
  Args:


# Validate correct tool call

In [17]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [18]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
tool_response = tool_node.invoke({"messages": messages})

In [19]:
from langchain_core.messages import ToolMessage

tool_message: ToolMessage = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: get_playlists

[{"id": "2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill "}, {"id": "75NW18NgdZuZeifrcjxKlZ", "name": "GVF"}, {"id": "5iyONtUO21O88xw8pBblwh", "name": "Now And Then"}, {"id": "4SDSUMg2HJcGFmHlupsU7U", "name": "Jazz classics"}, {"id": "5fBxLG2wLAPVpx83402rHl", "name": "Acdc"}, {"id": "17Yxm5hQbkbZwDcCTuiDXC", "name": "einaudi"}, {"id": "2X6GccWfIqPWO6VYx405nO", "name": "Eric Clapton Blues"}, {"id": "3smVXe8y8w8nt4lFH0HBkW", "name": "Alchemy: Dire Straits"}, {"id": "2ifRZBXnX7UmIPUlGet5WZ", "name": "Blues"}, {"id": "5nn2ePtd9gYEEDL3XnACCB", "name": "brazil"}, {"id": "6YAntoKNXuogPKdV26b6XE", "name": "CCR"}, {"id": "2VHkZvOMCkLF3B7TsKerKD", "name": "Chill"}, {"id": "508lbLx9XUkHv5Xb45hxTP", "name": "Classic Rock"}, {"id": "7zzDMq1sPwkZPYeqOQJ9qS", "name": "The New Four Seasons - Vivaldi Recomposed"}, {"id": "4l8WcaYlPkuVPXVGxMIc

# Add Tool Response to Messages

In [20]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [21]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-14 22:08:50,307 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  get_artists_from_playlist (call_UxVc0RrZuP2cEkNAHdFgNqa4)
 Call ID: call_UxVc0RrZuP2cEkNAHdFgNqa4
  Args:
    playlist_id: 01tk0aitEuGK0ajWCkzdKc


# Validate correct tool call

In [22]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'get_artists_from_playlist'")

# Add AI Tool Call to Messages

In [23]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [24]:
tool_response = tool_node.invoke({"messages": messages})
tool_message: ToolMessage = tool_response["messages"][0]
tool_message.pretty_print()

================================= Tool Message =================================
Name: get_artists_from_playlist

{"01aC2ikO4Xgb2LUpf9JfKp": "Gary Clark Jr.", "1iEaqWaYpKo9x0OrEq7Q7z": "Ayron Jones", "0RqtSIYZmd4fiBKVFqyIqD": "Thirty Seconds To Mars", "7jdFEYD2LTYjfwxOdlVjmc": "KALEO", "7Ln80lUS6He07XvHI8qqHH": "Arctic Monkeys", "4pejUc4iciQfgdX6OKulQn": "Queens of the Stone Age", "0Nrwy16xCPXG8AwkMbcVvo": "Black Pistol Fire", "1xKrH6GSh9CJh8nYwbqW7B": "Welshly Arms", "6vYg01ZFt1nREsUDMDPUYX": "The Record Company", "0bZCak2tcRMY1dzEIuwF42": "The Heavy", "4NpFxQe2UvRCAjto3JqlSl": "Greta Van Fleet", "26opZSJcXshCmCwxgZQmBc": "Whiskey Myers", "3xYXYzm9H3RzyQgBrYwIcx": "Colter Wall", "13ZEDW6vyBF12HYcZRr4EV": "Tyler Childers", "4YLtscXsxbVgi031ovDDdh": "Chris Stapleton", "5VPCIIfZPK8KPsgz4jmOEC": "The Blue Stones", "1l9I7G8J8AnMScWQwlNJ4M": "Goodbye June", "12Chz98pHFMPJEknJQMWvI": "Muse", "5HsS48kuvghKcNpwOaAvB5": "Samantha Fish", "7mnBLXK823vNxN3UWB7Gfz": "The Black Keys", "3mVWMgLc7bcyC

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [25]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Send Tool Result to LLM

In [26]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

2024-11-14 22:09:26,450 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  find_similar_artists (call_rS3hAJ5o6nnjMMVlcMS7ir7N)
 Call ID: call_rS3hAJ5o6nnjMMVlcMS7ir7N
  Args:
    artists: ['01aC2ikO4Xgb2LUpf9JfKp', '1iEaqWaYpKo9x0OrEq7Q7z', '0RqtSIYZmd4fiBKVFqyIqD', '7jdFEYD2LTYjfwxOdlVjmc', '7Ln80lUS6He07XvHI8qqHH', '4pejUc4iciQfgdX6OKulQn', '0Nrwy16xCPXG8AwkMbcVvo', '1xKrH6GSh9CJh8nYwbqW7B', '6vYg01ZFt1nREsUDMDPUYX', '0bZCak2tcRMY1dzEIuwF42']
  find_similar_artists (call_MaeruKdWgKnOZwaLSfPjb7WM)
 Call ID: call_MaeruKdWgKnOZwaLSfPjb7WM
  Args:
    artists: ['4NpFxQe2UvRCAjto3JqlSl', '26opZSJcXshCmCwxgZQmBc', '3xYXYzm9H3RzyQgBrYwIcx', '13ZEDW6vyBF12HYcZRr4EV', '4YLtscXsxbVgi031ovDDdh', '5VPCIIfZPK8KPsgz4jmOEC', '1l9I7G8J8AnMScWQwlNJ4M', '12Chz98pHFMPJEknJQMWvI', '5HsS48kuvghKcNpwOaAvB5', '7mnBLXK823vNxN3UWB7Gfz']
  find_similar_artists (call_batK6msAcMbXsLGvElrcqJgq)
 Call ID: call_batK6msAcMbXsLGvElrcqJgq
  Args:
    artists: ['3mVWMgLc7bcyCBtL2ymZwK', '2ziB7fzrX

# AI Tool Call (find_similar_artist())

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'find_similar_artists'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (find_similar_artists())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "filter_artists":
    raise ValueError("should be tool message calling 'filter_artists'")

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_top_tracks":
    raise ValueError("should be tool message calling 'find_top_tracks'")

# Tool Call

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
for tool_message in  tool_response["messages"]:
    tool_message.pretty_print()

In [ ]:
chat_prompt_template += tool_response["messages"]
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()